# Import All Functions


In [ ]:
from os import path

import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st

import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping
from sklearn.linear_model import LinearRegression 

#from time_series_conversion import build_timeseries, convert_price

# Connect to google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWihgLyUFpCahFTw_b0O1G5cRBem6hmLiTdzJi9viwHJiY8FJRefEKU
Mounted at /content/drive


# Price Generator




In [ ]:
TIME_STEPS = 24 # How far in the past you want to use past data as features
TARGET_TIME_STEPS = 12 # How far in the future you want to make predictions
#INP_COLS = [0, 1, 2, 3, 4]
INP_COLS = [1]
OUT_COL = 1
N_ITER = 500


def build_timeseries(mat, input_cols, output_col, time_steps, target_time_steps):
    # input_cols is a list of columns that would act as input columns
    # output_col is the column that would act as output column
    # total number of time-series samples = len(mat) - time_steps - target_time_steps + 1
    dim_0 = mat.shape[0] - time_steps - target_time_steps + 1
    dim_1 = len(input_cols)
    
    x = np.zeros((dim_0, time_steps))
    y = np.zeros((dim_0,))
    
    for i in range(dim_0):  
        x[i] = mat[i: time_steps + i]
        y[i] = mat[time_steps + target_time_steps + i - 1]

    return x, y

def convert_price(pred, sc, feature_num, out_col):
# Return predicted price for given input
    #pred_reverted = np.zeros(shape=(pred.shape[0], 1))
    #pred_reverted[:, 0] = pred
    pred_reverted = sc.inverse_transform(pred.reshape(-1, 1))
    predicted_price = pred_reverted[:]
    return predicted_price

In [ ]:
def lstm_price_generator(current_data):
  sc = MinMaxScaler(feature_range = (0, 1))
  normalized_data = sc.fit_transform(current_data.values.reshape(-1, 1))
  normalized_data = normalized_data.ravel()
  normalized_data = pd.Series(list(normalized_data))
  
  current_data_val = current_data.iloc[int(0.6*train_len):train_len]
  current_data_test = current_data.iloc[train_len:]
  #Build time series model for linear regression
  x_train, y_train = build_timeseries(mat=normalized_data.iloc[:int(0.6*train_len)].values, input_cols=INP_COLS, output_col=OUT_COL, 
                                          time_steps=TIME_STEPS, target_time_steps=TARGET_TIME_STEPS)
  x_val, y_val = build_timeseries(mat=normalized_data.iloc[int(0.6*train_len) -(TIME_STEPS + TARGET_TIME_STEPS - 1):train_len].values, input_cols=INP_COLS, output_col=OUT_COL, 
                                          time_steps=TIME_STEPS, target_time_steps=TARGET_TIME_STEPS)
  x_test, y_test = build_timeseries(mat=normalized_data.iloc[int(train_len) -(TIME_STEPS + TARGET_TIME_STEPS - 1):].values, input_cols=INP_COLS, output_col=OUT_COL, 
                                          time_steps=TIME_STEPS, target_time_steps=TARGET_TIME_STEPS)
  print(y_train)
  inputs = Input(shape=(TIME_STEPS, 1))
  hidden = LSTM(64, dropout=0.01, recurrent_dropout=0.05)(inputs)
  hidden = Dense(32, activation='relu')(hidden)
  hidden = Dropout(0.1)(hidden) 
  outputs = Dense(1)(hidden)
  model = Model(inputs, outputs)
  # Compiling the RNN
  model.compile(optimizer = 'adam', loss = 'mean_squared_error')
  model.summary()
  NUM_EPOCHS = 500
  early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=20)

  # Fitting the RNN to the Training set
  history = model.fit(
              x=x_train, 
              y=y_train,
              epochs=NUM_EPOCHS,
              batch_size=64,
              validation_data=(x_val, y_val), 
              callbacks=[early_stopping_monitor]
  )

  pred_val = model.predict(x_val).flatten()
  predicted_price_val = convert_price(pred_val, sc, 1, OUT_COL)
  try:
    inherent_noice = mean_squared_error(current_data_val, predicted_price_val)
  except:
    inherent_noice = 0
  pred_do = np.stack([model(x_test, training=True) for _ in range(N_ITER)])
  pred_do = pred_do.reshape((N_ITER, testing_num))
  # .,,,,.print(predicted_price_val)
  # Convert these predictions into price
  for row in range(pred_do.shape[0]):
    #print(convert_price(pred_do[row, :], sc, 1, OUT_COL))
    pred_do[row, :] = convert_price(pred_do[row, :], sc, 1, OUT_COL).T

  current_average = 0
  current_std = 0
  average_predicted_price = []
  std_predicted_price = []
  for i in pred_do.T:
    if not math.isnan(i.mean()):
      average_predicted_price.append(i.mean())
      current_average = i.mean()
      current_std = (i.std() + inherent_noice) ** 0.5
      std_predicted_price.append(current_std)
    else :
      average_predicted_price.append(current_average)
      std_predicted_price.append(current_std)

  return average_predicted_price, std_predicted_price

# Cryptocurrency

## BTC-USD

In [ ]:
btc_data = pd.read_csv('/content/drive/My Drive/lstm_stock_predictions/BTC-USD.csv', index_col=0, parse_dates=True)
total = len(btc_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(btc_data))
train_len
predicted_btc_price, predicted_btc_std = lstm_price_generator(btc_data['Close'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
12/12 [==============================] - 4s 89ms/step - loss: 0.0051 - val_loss: 3.1706e-04
Epoch 2/500
12/12

In [ ]:
test_price = btc_data[train_len:] 
test_price['Predicted'] = predicted_btc_price
test_price['Predicted_std'] = predicted_btc_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/BTC-USD_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## USD - ETH

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/ETH-USD.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close'])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
12/12 [==============================] - 4s 88ms/step - loss: 0.0061 - val_loss: 5.1700e-04
Epoch 2/500
12/12

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/ETH-USD_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## USD - DOGE

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/DOGE-USD.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close'])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
12/12 [==============================] - 4s 90ms/step - loss: 1.9181e-05 - val_loss: 3.4484e-06
Epoch 2/500
1

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/DOGE-USD_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## USD - XRP

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/XRP-USD.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close'])

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
12/12 [==============================] - 4s 86ms/step - loss: 0.0282 - val_loss: 3.4312e-04
Epoch 2/500
12/12

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/XRP-USD_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## USD - LTC

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/LTC-USD.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close'])

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
12/12 [==============================] - 4s 87ms/step - loss: 0.0462 - val_loss: 0.0037
Epoch 2/500
12/12 [==

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/LTC-USD_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##USD - BCH

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/BCH-USD.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_14 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
9/9 [==============================] - 4s 119ms/step - loss: 0.0503 - val_loss: 0.0046
Epoch 2/500
9/9 [=====

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/BCH-USD_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## USD - ADA

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/ADA-USD.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_16 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
9/9 [==============================] - 4s 109ms/step - loss: 0.0086 - val_loss: nan
Epoch 2/500
9/9 [========

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/ADA-USD_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## USD - BNB

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/BNB-USD.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_18 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
9/9 [==============================] - 4s 112ms/step - loss: 9.6557e-05 - val_loss: 3.5047e-04
Epoch 2/500
9/

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/BNB-USD_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# S&P 500

## AAPL

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/AAPL.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.00551018 0.00756534 0.00923599 ... 0.16494547 0.16244354 0.151962  ]
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/AAPL_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##FB

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/FB.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_22 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
15/15 [==============================] - 4s 78ms/step - loss: 0.0170 - val_loss: 0.0039
Epoch 2/500
15/15 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/FB_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##TSLA

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/TSLA.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_24 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 71ms/step - loss: 6.0580e-04 - val_loss: 8.2940e-04
Epoch 2/500


In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/TSLA_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##GOOGL

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/GOOGL.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_26 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 73ms/step - loss: 0.0080 - val_loss: 0.0082
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/GOOGL_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##BRK-B

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/BRK-B.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_28 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 71ms/step - loss: 0.0217 - val_loss: 0.0034
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/BRK-B_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##JPM

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/JPM.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_30 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_15 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 73ms/step - loss: 0.0185 - val_loss: 0.0052
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/JPM_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##JNJ

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/JNJ.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_32 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_16 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 72ms/step - loss: 0.0329 - val_loss: 0.0109
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/JNJ_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##SPY

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/SPY.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_34 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_17 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 70ms/step - loss: 0.0155 - val_loss: 0.0050
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/SPY_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##JNJ

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/JNJ.csv', index_col=0, parse_dates=True)
raw_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-05-19,66.529999,66.529999,65.849998,66.389999,49.643864,9685100
2011-05-20,66.250000,66.330002,65.279999,65.690002,49.120430,12367000
2011-05-23,65.190002,65.690002,65.080002,65.559998,49.023212,12020200
2011-05-24,65.739998,66.230003,65.500000,65.779999,49.187725,11017700
2011-05-25,66.290001,66.589996,66.070000,66.290001,49.569096,14767100
...,...,...,...,...,...,...
2021-05-12,168.539993,169.800003,167.949997,168.199997,168.199997,6758300
2021-05-13,166.970001,170.699997,166.970001,169.960007,169.960007,8018600
2021-05-14,171.479996,171.610001,170.169998,170.220001,170.220001,5797700


In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/JNJ.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.06410951 0.06193325 0.06737392 ... 0.35201304 0.35845125 0.35944865]
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/JNJ_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# HSI 

## 0700 (Tencent)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/0700.HK.csv', index_col=0, parse_dates=True)
raw_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-07-05,42.919998,43.160000,42.680000,43.000000,41.858170,11164345.0
2011-07-06,42.840000,43.520000,42.439999,42.919998,41.780296,15582130.0
2011-07-07,43.200001,43.599998,42.759998,42.959999,41.819229,13562530.0
2011-07-08,43.599998,43.599998,42.119999,42.520000,41.390919,23646460.0
2011-07-11,42.320000,42.400002,41.040001,41.400002,40.300659,22207165.0
...,...,...,...,...,...,...
2021-06-25,588.500000,600.000000,584.000000,598.500000,598.500000,19252093.0
2021-06-28,603.000000,606.000000,592.000000,595.500000,595.500000,11485959.0
2021-06-29,600.000000,601.000000,588.000000,590.500000,590.500000,14714259.0


In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/0700.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[8.10891304e-03 8.02755134e-03 8.13603451e-03 8.19027609e-03
 9.32931550e-03 9.89884063e-03 1.07938044e-02 1.14989274e-02
 1.14175657e-02 1.00344398e-02 1.02242847e-02 1.15260489e-02
 1.11463632e-02 1.06582066e-02 8.46147318e-03 9.03099830e-03
 7.97330975e-03 8.97675672e-03 7.64787108e-03 7.24107071e-03
 6.29186533e-03 3.09169176e-03 3.74257316e-03 2.90185095e-03
 4.01377567e-03 5.47826459e-03 5.15282050e-03 1.03056166e-03
 1.08483172e-04 4.82737641e-03 6.96986278e-03 6.34610692e-03
 6.75290729e-03 7.62075097e-03 7.48515175e-03 7.43091016e-03
 8.86827897e-03 6.02066418e-03 6.18338216e-03 4.82737641e-03
 4.90873946e-03 7.40379005e-03 8.59707646e-03 8.65131263e-03
 1.07938044e-02 1.13362094e-02 1.02785209e-02 8.75979580e-03
 1.04954818e-02 9.27507934e-03 9.98020369e-03 7.91907359e-03
 5.34266130e-03 5.77658179e-03 3.85105362e-03 3.39001438e-03
 4.01377567e-03 3.28153121e-03 2.92897242e-03 2.35945001e-03
 1.35600575e-03 2.27808695e-03 3.68833429e-03 2.49504787e-03
 3.14593334e-03 1.572966

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/0700_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 1398 (ICBC)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/1398.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_40 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_20 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 70ms/step - loss: nan - val_loss: nan
Epoch 2/500
16/16 [=======

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/1398_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 4333 (CISCO-T)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/4333.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_42 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_21 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 73ms/step - loss: nan - val_loss: nan
Epoch 2/500
16/16 [=======

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/4333_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 3968 (CM Bank)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/3968.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_44 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_22 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 71ms/step - loss: nan - val_loss: nan
Epoch 2/500
16/16 [=======

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/3968_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 939 (CCB)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/0939.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_46 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_23 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 72ms/step - loss: nan - val_loss: nan
Epoch 2/500
16/16 [=======

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/0939_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 2318 (Ping An)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/2318.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_48 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_24 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 71ms/step - loss: nan - val_loss: nan
Epoch 2/500
16/16 [=======

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/2318_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 1288 (ABC)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/1288.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_50 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_25 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 69ms/step - loss: nan - val_loss: nan
Epoch 2/500
16/16 [=======

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/1288_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 1299 (AIA)

In [ ]:
raw_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-07-05,27.500000,27.799999,27.299999,27.600000,24.136076,21992190.0
2011-07-06,27.750000,28.000000,27.200001,27.299999,23.873728,32105343.0
2011-07-07,27.200001,27.950001,27.200001,27.750000,24.267256,35830557.0
2011-07-08,27.900000,28.150000,27.799999,27.950001,24.442152,23433261.0
2011-07-11,27.850000,27.900000,27.400000,27.450001,24.004904,11763659.0
...,...,...,...,...,...,...
2021-06-25,98.500000,98.500000,96.949997,97.150002,97.150002,14431558.0
2021-06-28,97.599998,97.800003,95.199997,95.800003,95.800003,15169745.0
2021-06-29,96.550003,96.550003,95.500000,95.699997,95.699997,14331011.0


In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/1299.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.0647482  0.06306203 0.06643434 0.06137589 0.06531024 0.07992354
 0.08329585 0.0804856  0.07317896 0.06868253 0.0703687  0.0782374
 0.07542715 0.06868253 0.05350719 0.04114207 0.05294513 0.05406923
 0.04901077 0.05069694 0.04226619 0.01866006 0.02821493 0.02652875
 0.03102518 0.02484261 0.02652875 0.01866006 0.         0.02203236
 0.03383543 0.03046312 0.03889387 0.04844873 0.05013488 0.04451438
 0.04058001 0.03046312 0.03608362 0.03664568 0.03439747 0.04339026
 0.04058001 0.04226619 0.04451438 0.04676258 0.04676258 0.04788669
 0.04563848 0.04001797 0.04901077 0.05069694 0.05463127 0.05519333
 0.04451438 0.04507644 0.05182102 0.04620052 0.04170412 0.04282823
 0.04170412 0.03495951 0.03833182 0.03608362 0.04282823 0.04170412
 0.03945593 0.04226619 0.04058001 0.04620052 0.04395233 0.04620052
 0.04395233 0.04844873 0.04563848 0.04001797 0.04282823 0.03889387
 0.04058001 0.03158722 0.03271132 0.03158722 0.03889387 0.04451438
 0.04451438 0.04732463        nan 0.04732463 0.04676258 0.04844

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/1299_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 0857 (Petrochina)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/0857.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_54 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_27 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 73ms/step - loss: nan - val_loss: nan
Epoch 2/500
16/16 [=======

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/0857_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 3988 (Bank of China)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/3988.HK.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_56 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_28 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 72ms/step - loss: nan - val_loss: nan
Epoch 2/500
16/16 [=======

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/3988_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# SSE

## 601628 (China Life Insurance)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/601628.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_58 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_29 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 72ms/step - loss: 0.0476 - val_loss: 0.0040
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/601628_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 601857 (Petrochina)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/601857.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_30 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_60 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_30 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 71ms/step - loss: 0.1531 - val_loss: 0.0141
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/601857_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 601318 (Ping An Insurance)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/601318.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_62 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_31 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 72ms/step - loss: 0.0111 - val_loss: 0.0133
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/601318_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 601939 (China Construction Bank)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/601939.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_32 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_64 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_32 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 71ms/step - loss: 0.0273 - val_loss: 0.0066
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/601939_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 600036 (China Merchants Bank)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/600036.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_33 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_66 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_33 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_67 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 70ms/step - loss: 0.0047 - val_loss: 0.0181
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/600036_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 601398 (Industrial and Commercial Bank of China)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/601398.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_68 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_34 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 69ms/step - loss: 0.0298 - val_loss: 0.0153
Epoch 2/500
16/16 [=

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/601398_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 600519 (Kweichow Moutai)

In [ ]:
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/600519.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

Model: "model_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_70 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_35 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 4s 71ms/step - loss: 2.1620e-04 - val_loss: 0.0036
Epoch 2/500
16/1

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/600519_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#2 week minute data


### Parse the raw_data


In [ ]:
raw_data = pd.read_csv('/content/drive/My Drive/lstm_stock_predictions/crypto_2_weeks.csv', index_col=0, parse_dates=True)
crypto_data = {}
for name in raw_data.index.unique() :
  print(name)
  crypto_data[name] = pd.DataFrame()
  crypto_data[name]['Time'] = pd.to_datetime(raw_data.loc[name]['TIME'], format='%d-%m-%y %H:%M:%S')
  crypto_data[name]['Close'] = (raw_data.loc[name]['ASK'] + raw_data.loc[name]['BID'])/2
  crypto_data[name] = crypto_data[name].set_index('Time')
  crypto_data[name] = crypto_data[name].sort_index()

YFI/USD
UNI/USD
LINK/USD
ADA/USD
DOT/USD
DOG/USD
UST/USD
LTC/USD
ETH/USD
BTC/USD


##YFI-USD

In [ ]:
raw_data = crypto_data['YFI/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.53019713 0.49638734 0.46324511 ... 0.33660567 0.33876541 0.33951151]
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                16896     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
416/416 [==============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/YFI-USD_2_Week_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## UNI-USD

In [ ]:
raw_data = crypto_data['UNI/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.55043115 0.50759388 0.43115438 ... 0.30097357 0.30130737 0.30114047]
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
416/416 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/UNI-USD_2_Week_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##LINK-USD

In [ ]:
raw_data = crypto_data['LINK/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.51891201 0.49626553 0.45776251 ... 0.27223709 0.27475337 0.27611136]
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
416/416 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/LINK-USD_2_Week_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##ADA-USD

In [ ]:
raw_data = crypto_data['ADA/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.40297648 0.39644743 0.34008641 ... 0.33307729 0.33346135 0.33528565]
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
416/416 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/ADA-USD_2_Week_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##DOT-USD

In [ ]:
raw_data = crypto_data['DOT/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.12868375 0.11595443 0.07765952 ... 0.19088624 0.19190244 0.19179548]
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
416/416 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/DOT-USD_2_Week_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##DOG-USD

In [ ]:
raw_data = crypto_data['DOG/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.04082    0.02507566 0.01469952 ... 0.16659461 0.16814382 0.16223519]
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
416/416 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/DOG-USD_2_Week_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##UST-USD

In [ ]:
raw_data = crypto_data['UST/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/UST-USD_2_Week_Predicted.csv')

##LTC-USD

In [ ]:
raw_data = crypto_data['LTC/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/LTC-USD_2_Week_Predicted.csv')

## ETH-USD

In [ ]:
train_len = int(0.7 * len(crypto_data['ETH/USD']))
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/600519.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/ETH-USD_2_Week_Predicted.csv')

##BTC-USD

In [ ]:
train_len = int(0.7 * len(crypto_data['BTC/USD']))
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/600519.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/BTC-USD_2_Week_Predicted.csv')

#1 month minute data


### Parse the raw_data


In [ ]:
raw_data = pd.read_csv('/content/drive/My Drive/lstm_stock_predictions/crypto_1_month.csv', index_col=0, parse_dates=True)
crypto_data = {}
for name in raw_data.index.unique() :
  print(name)
  crypto_data[name] = pd.DataFrame()
  crypto_data[name]['Time'] = pd.to_datetime(raw_data.loc[name]['TIME'], format='%d-%m-%y %H:%M:%S')
  crypto_data[name]['Close'] = (raw_data.loc[name]['ASK'] + raw_data.loc[name]['BID'])/2
  crypto_data[name] = crypto_data[name].set_index('Time')
  crypto_data[name] = crypto_data[name].sort_index()

UST/USD
BTC/USD
ETH/USD
LTC/BTC
YFI/USD
UNI/USD
LINK/USD
ADA/USD
DOT/USD
DOG/USD


##YFI-USD

In [ ]:
raw_data = crypto_data['YFI/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.80066894 0.77623714 0.77784189 ... 0.24371399 0.24370498 0.24347058]
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                16896     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
433/433 [==============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/YFI-USD_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## UNI-USD

In [ ]:
raw_data = crypto_data['UNI/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.87831648 0.85412776 0.86566717 ... 0.32035517 0.32072513 0.3206899 ]
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
433/433 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/UNI-USD_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##LINK-USD

In [ ]:
raw_data = crypto_data['LINK/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.89406559 0.87808411 0.87767473 ... 0.2264962  0.2266694  0.22696856]
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
433/433 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/LINK-USD_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##ADA-USD

In [ ]:
raw_data = crypto_data['ADA/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.55201367 0.52931342 0.5346634  ... 0.29551196 0.2968123  0.29599495]
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                16896     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
433/433 [==============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/ADA-USD_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##DOT-USD

In [ ]:
raw_data = crypto_data['DOT/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.74651095 0.71570803 0.72366423 ... 0.16773723 0.16827737 0.16820438]
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
433/433 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/DOT-USD_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##DOG-USD

In [ ]:
raw_data = crypto_data['DOG/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.84035655 0.89305003 0.93231093 ... 0.18372305 0.18417184 0.18428017]
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
433/433 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/DOG-USD_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##UST-USD

In [ ]:
raw_data = crypto_data['UST/USD']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.37200957 0.38397129 0.37200957 ... 0.43181818 0.43181818 0.43181818]
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
433/433 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/UST-USD_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##LTC-BTC

In [ ]:
raw_data = crypto_data['LTC/BTC']
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.78688423 0.77938288 0.79311731 ... 0.27744181 0.27699327 0.27702421]
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 24, 1)]           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
433/433 [============================

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/LTC-BTC_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## ETH-USD

In [ ]:
train_len = int(0.7 * len(crypto_data['ETH/USD']))
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/600519.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.01920395 0.01920395 0.02042869 0.01894278 0.0215814  0.02196263
 0.02379075 0.02280014 0.02430706 0.02318438 0.02344554 0.02136827
 0.0215904  0.02126921 0.02216075 0.02178552 0.02172849 0.02341252
 0.02186057 0.02181254 0.02260202 0.0232204  0.02275511 0.02330445
 0.02395885 0.02410594 0.02461325 0.02406182 0.02452079 0.02399577
 0.0238769  0.02488072 0.02487081 0.02577886 0.02475854 0.02503921
 0.02459674 0.02522413 0.02648219 0.02644257 0.02678928 0.0279582
 0.02889927 0.02803084 0.02800112 0.02825207 0.02938467 0.02903465
 0.02794169 0.02719213 0.02741667 0.02847001 0.02785914 0.02671334
 0.02718883 0.02870776 0.03167957 0.03236309 0.03121729 0.03002526
 0.03007809 0.02953326 0.0300946  0.02824878 0.0284535  0.02874077
 0.02927571 0.03107861 0.03186779 0.0317192  0.03170269 0.03061632
 0.0307418  0.02922617 0.02781951 0.02933184 0.02876389 0.02916343
 0.02736384 0.02614869 0.026327   0.0277898  0.02772375 0.02710298
 0.02826529 0.02605293 0.0248609  0.0229028  0.02334527 0.02257

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/ETH-USD_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##BTC-USD

In [ ]:
train_len = int(0.7 * len(crypto_data['BTC/USD']))
raw_data = pd.read_csv(r'/content/drive/My Drive/lstm_stock_predictions/600519.SS.csv', index_col=0, parse_dates=True)
total = len(raw_data)
training_num = int(total * 0.7)
testing_num = total - training_num 
train_len = int(0.7 * len(raw_data))
train_len
predicted_price, predicted_price_std = lstm_price_generator(raw_data['Close']) 

[0.01920395 0.01920395 0.02042869 0.01894278 0.0215814  0.02196263
 0.02379075 0.02280014 0.02430706 0.02318438 0.02344554 0.02136827
 0.0215904  0.02126921 0.02216075 0.02178552 0.02172849 0.02341252
 0.02186057 0.02181254 0.02260202 0.0232204  0.02275511 0.02330445
 0.02395885 0.02410594 0.02461325 0.02406182 0.02452079 0.02399577
 0.0238769  0.02488072 0.02487081 0.02577886 0.02475854 0.02503921
 0.02459674 0.02522413 0.02648219 0.02644257 0.02678928 0.0279582
 0.02889927 0.02803084 0.02800112 0.02825207 0.02938467 0.02903465
 0.02794169 0.02719213 0.02741667 0.02847001 0.02785914 0.02671334
 0.02718883 0.02870776 0.03167957 0.03236309 0.03121729 0.03002526
 0.03007809 0.02953326 0.0300946  0.02824878 0.0284535  0.02874077
 0.02927571 0.03107861 0.03186779 0.0317192  0.03170269 0.03061632
 0.0307418  0.02922617 0.02781951 0.02933184 0.02876389 0.02916343
 0.02736384 0.02614869 0.026327   0.0277898  0.02772375 0.02710298
 0.02826529 0.02605293 0.0248609  0.0229028  0.02334527 0.02257

In [ ]:
test_price = raw_data[train_len:] 
test_price['Predicted'] = predicted_price
test_price['Predicted_std'] = predicted_price_std
test_price[['Predicted','Predicted_std']].to_csv('/content/drive/My Drive/lstm_stock_predictions/BTC-USD_1_Month_Predicted.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
